In [ ]:
"""
Extract gridMET variables 
Google Earth Engine (GEE) Python API
Author: maxwell.cook@colorado.edu
"""

import os, sys
import ee
import geemap
import time

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

ee.Authenticate()
ee.Initialize(project='jfsp-aspen')

print("Success !")

In [ ]:
# Load fire perimeters
fires = ee.FeatureCollection('projects/jfsp-aspen/assets/nifc_aspenfires_2018to2023')
print(f"Number of aspen fires: {fires.size().getInfo()}")
print(fires.first().propertyNames().getInfo())

In [ ]:
# Load the gridmet image collection

In [ ]:
gridmet = ee.ImageCollection('IDAHO_EPSCOR/GRIDMET')
print(f"gridMET bands available for analysis:\n\n{gridmet.first().bandNames().getInfo()}")

In [ ]:
# calculate the 15 year average
mean15 = gridmet.filter(ee.Filter.calendarRange(2008,2024,'year')).mean() 
gridmet.first().bandNames().getInfo()

In [ ]:
# Load the AFD 

In [ ]:
grid = ee.FeatureCollection('projects/jfsp-aspen/assets/viirs_snpp_jpss1_afd_aspenfires_gridstats')
print(f"Number of gridcells: {grid.size().getInfo()}")
print(grid.first().propertyNames().getInfo())

In [ ]:
# get a list of unique active fire days
fire_days = grid.aggregate_array('max_date').getInfo()
fire_days = set(fire_days)
print(f"Number of 'fire days': {len(fire_days)}")

In [ ]:
# map over the list of unique dates
def gridmet_summary(fireID):
    """ calculate the daily gridmet for active fire detections """
    # grab the fire id
    # filter obs. to the day of burning
    # get the unique dates of max FRP
    fire_grid = grid.filter(ee.Filter.eq('Fire_ID', fireID))
    fire_days = ee.List(fire_grid.aggregate_array('max_date')).distinct()
    
    def get_daily(day):
        """ calculate the daily gridmet summary"""
        # filter to this day's detections
        # retrieve the date information for filtering
        # get bounds for all grids in the same day
        grids_day = fire_grid.filter(ee.Filter.eq('max_date', day))
        bounds = grids_day.geometry().bounds() # gather the bounds for this days detections
        max_frp_day = ee.Date(day) # the day of maximum FRP
        prev_day = max_frp_day.advance(-1,'days') # previous day for calculating the mean
        
        # calculate the gridmet mean
        # calculation the deviation from the 15-year average
        gridmet_ = gridmet.filterDate(prev_day, max_frp_day).select(attrs).mean()
        anomaly = gridmet_.subtract(mean15.select(attrs)).rename([atr+"_dv" for atr in attrs])
        combined = gridmet_.addBands(anomaly) # combine the bands
        
        # run the reduction
        stats = combined.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=bounds,
            scale=4000,
            bestEffort=True
        )

        return ee.Feature(None, stats.set("max_date", day))

    results = fire_days.map(get_daily)
    return ee.FeatureCollection(results).set('Fire_ID', fireID)

print("Function ready !")

In [ ]:
# list of gridmet attributes
attrs = ['fm1000', 'vpd', 'erc', 'vs']

# get a list of fire names
fire_ids = grid.aggregate_array('Fire_ID').distinct()
print(f"Processing {fire_ids.size().getInfo()} fires.")

# run the gridmet summary for fires
def map_fires(fireID):
    return gridmet_summary(fireID)

grid_stats = ee.FeatureCollection(
    fire_ids.map(map_fires).flatten()
)

print("Number of features:", grid_stats.first().getInfo())
print("Submitted !")

In [ ]:
grid_stats.first().getInfo()

In [ ]:
# Export the table. 
fire_stats = fire_stats.map(lambda ftr: ftr.setGeometry(None)) # drop geometry column

export_task = ee.batch.Export.table.toDrive(
    collection=fire_stats,
    description='vn14img_gridmet',
    fileFormat='CSV', 
    fileNamePrefix='vn14img_gridmet',
    folder='GRIDMET'
)

# Start the export task
export_task.start()
print("Export to Earth Engine Asset started!")
# Monitor the task until it's finished
monitor_export(export_task, 240) 